### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

large dataset

In [2]:
(load-large-dataset "../../data-sorted-cleaned/data-CRSP-sorted-cleaned.csv" "main" add-aprc-by-date)
(set-main "main")

true

### Initialise portfolio （Go back here everytime you want to restart.）

In [3]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

In [6]:
;; (first (get (deref dataset-col) "main"))
;; (get-date)
;; (get-line)
;; (def x [])
;; (for [i (range 1 30)]
;;     (do(def x (get-prev-n-days :PRC 10 "A" x))
;;         (next-date)
;;         (println x)
;;     ))
;; (conj [2344 235 234 ] {:akj 24})
(get-date)

"2013-03-25"

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 50 cross below the MA 200 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [4]:
(def MA50-vec-aapl [])
(def MA200-vec-aapl [])
(while (< (compare (get-date) "2015-12-24") 0)
    (do
      (def tics (deref available-tics)) ;20 ms
      (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl))
      (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl))
      (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f ))
      (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
        (if (> MA50 MA200)
          (order "AAPL" 0.1 :print false) 
          (order "AAPL" 0 :remaining true))))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
        (if (> MA50 MA200)
          (order "F" 1  :print false) 
          (order "F" 0 :remaining true )))
      (update-eval-report (get-date))
      (next-date))
(end-order)

[]

large dataset

In [29]:
(def MA50-vec-aapl [])
(def MA200-vec-aapl [])
(def MA50-vec-f [])
(def MA200-vec-f [])
(while (< (compare (get-date) "2015-12-24") 0)
    (do
      ;(def tics (deref available-tics)) ;20 ms
      (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl))
      (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl))
      (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f ))
      (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
        (if (> MA50 MA200)
          (order-lazy "AAPL" 0.1 :print false) 
          (order-lazy "AAPL" 0 :remaining true))))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
        (if (> MA50 MA200)
          (order-lazy "F" 1  :print false) 
          (order-lazy "F" 0 :remaining true )))
      (update-eval-report (get-date))
      (next-date))
(end-order)

nil

In [11]:
;; (count(deref order-record))
;; (deref portfolio)
(doseq [[ticker] (deref portfolio)]
    (if (not= ticker :cash)      
      (order ticker 0 :remaining true)
    )
  )

The order request 2016-12-30 | AAPL | 0 fails.
Failure reason: null.


nil

In [5]:
;(pprint/print-table (take 200 (deref portfolio-value)))
(take 10 (deref order-record))

({:date "1981-02-27", :tic "AAPL", :price 26.625, :aprc "26.39", :quantity 0.1} {:date "1981-02-27", :tic "F", :price 19.625, :aprc "103.35", :quantity 1} {:date "1981-03-02", :tic "AAPL", :price 26.6875, :aprc "26.42", :quantity 0.1} {:date "1981-03-02", :tic "F", :price 20.125, :aprc "104.49", :quantity 1} {:date "1981-03-03", :tic "AAPL", :price 26.3125, :aprc "26.26", :quantity 0.1} {:date "1981-03-03", :tic "F", :price 20.25, :aprc "104.77", :quantity 1} {:date "1981-03-04", :tic "AAPL", :price 26.0625, :aprc "26.15", :quantity 0.1} {:date "1981-03-04", :tic "F", :price 20.25, :aprc "104.77", :quantity 1} {:date "1981-03-05", :tic "AAPL", :price 25.9375, :aprc "26.09", :quantity 0.1} {:date "1981-03-05", :tic "F", :price 20.125, :aprc "104.49", :quantity 1})

### Check portfolio record

In [6]:
;; view final portfolio
(view-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A |  3581.11 |


nil

In [7]:
;; view portfolio value and return
(view-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage | :margin |
|------------+------------+------------+----------+-------+-----------+---------|
| 1980-12-15 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1981-02-27 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1981-03-02 |   $1001.14 |      0.00% |    0.05% | $0.00 |      0.00 |   0.00% |
| 1981-03-03 |   $1001.67 |      0.00% |    0.07% | $0.00 |      0.00 |   0.00% |
| 1981-03-04 |   $1001.64 |      0.00% |    0.07% | $0.00 |      0.00 |   0.00% |
| 1981-03-05 |   $1000.49 |      0.00% |    0.02% | $0.00 |      0.00 |   0.00% |
| 1981-03-06 |   $1011.57 |      0.00% |    0.50% | $0.00 |      0.00 |   0.00% |
| 1981-03-09 |   $1028.79 |      0.00% |    1.23% | $0.00 |      0.00 |   0.00% |
| 1981-03-10 |   $1017.20 |      0.00% |    0.74% | $0.00 |      0.00 |   0.00% |
| 1981-03-11 |   $1019.04 |      0.00% |    0.82% | $0.00 |      0.00 |   0.00% |


nil

### Generate evaluation report

In [7]:
(eval-report 434)
;; (update-eval-report)
;; (count(deref eval-report-data))
;; (deref dailyret-record)


|      :date | :tot-value |     :vol |                 :sharpe | :pnl-pt |
|------------+------------+----------+-------------------------+---------|
| 1981-02-26 |      $1000 |  0.0000% |                 0.0000% |      $0 |
| 1981-02-27 |      $1001 |  0.0285% |                 1.7362% |      $0 |
| 1981-03-02 |      $1001 |  0.0122% |                 5.9335% |      $0 |
| 1981-03-03 |      $1001 |  0.0000% |                 0.0000% |      $0 |
| 1981-03-04 |      $1000 |  0.0199% |                 1.0642% |      $0 |
| 1981-03-05 |      $1011 |  0.1814% |                 2.7529% |      $0 |
| 1981-03-06 |      $1028 |  0.2676% |                 4.6055% |      $2 |
| 1981-03-09 |      $1017 |  0.1588% |                 4.6637% |      $1 |
| 1981-03-10 |      $1019 |  0.0291% |                28.1887% |      $1 |
| 1981-03-11 |      $1026 |  0.0059% |               142.8007% |      $1 |
| 1981-03-12 |      $1028 |  0.0055% |               155.8741% |      $1 |
| 1981-03-13 |      $103

| 1981-07-31 |       $801 |  2.0880% |               -84.4380% |     $-1 |
| 1981-08-03 |       $832 |  2.0398% |               -75.6972% |     $-1 |
| 1981-08-04 |       $712 | 10.2937% |               -25.4924% |     $-1 |
| 1981-08-05 |       $712 |  0.0000% |                 0.0000% |     $-1 |
| 1981-08-06 |       $681 |  3.0895% |               -95.6151% |     $-1 |
| 1981-08-07 |       $618 |  7.4469% |               -50.3913% |     $-2 |
| 1981-08-10 |       $586 |  4.3140% |               -97.7555% |     $-2 |
| 1981-08-11 |       $618 |  4.1437% |               -90.9563% |     $-2 |
| 1981-08-12 |       $683 |  7.0038% |               -42.9506% |     $-1 |
| 1981-08-13 |       $683 |  0.0000% |                 0.0000% |     $-1 |
| 1981-08-14 |       $650 |  3.6785% |               -92.7320% |     $-2 |
| 1981-08-17 |       $583 |  9.0280% |               -48.7839% |     $-2 |
| 1981-08-18 |       $583 |  0.0000% | -1689988096198580.0000% |     $-2 |
| 1981-08-19 |       $549

| 1982-01-07 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-08 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-11 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-12 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-13 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-14 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-15 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-18 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-19 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-20 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-21 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-22 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-25 |       $446 |  9.1473% |              -103.5007% |     $-3 |
| 1982-01-26 |       $446

| 1982-06-15 |       $484 | 19.7274% |               -48.5475% |     $-2 |
| 1982-06-16 |       $484 |  7.7769% |              -123.1477% |     $-2 |
| 1982-06-17 |       $470 |  8.5562% |              -117.7430% |     $-2 |
| 1982-06-18 |       $456 |  8.6795% |              -122.3232% |     $-2 |
| 1982-06-21 |       $498 | 13.3145% |               -68.6373% |     $-2 |
| 1982-06-22 |       $640 | 24.9738% |               -23.5715% |     $-1 |
| 1982-06-23 |       $626 |  7.9683% |               -77.6529% |     $-1 |
| 1982-06-24 |       $538 | 18.7649% |               -45.5899% |     $-1 |
| 1982-06-25 |       $568 |  9.6425% |               -80.3339% |     $-1 |
| 1982-06-28 |       $598 |  9.3263% |               -75.1125% |     $-1 |
| 1982-06-29 |       $598 |  7.5935% |               -92.2536% |     $-1 |
| 1982-06-30 |       $504 | 22.4368% |               -44.3573% |     $-2 |
| 1982-07-01 |       $373 | 50.6630% |               -33.4631% |     $-2 |
| 1982-07-02 |       $356

nil

In [9]:
(take 1 (deref eval-report-data))

({:date "1981-02-26", :tot-val "$1000", :vol "0.00%", :sharpe "0.00%", :pnl-pt "$0"})

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)